In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.vivanda.com.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
27


In [3]:
# file_path = 'vivanda_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [4]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [5]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [6]:
df.query("id == 952")

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id
640,952,Zapatillas Outdoor para Mujer,False,https://www.vivanda.com.pe/moda/zapatillas-muj...,0,Zapatillas Outdoor para Mujer,,908


In [7]:
file_name_categories_csv = f'vivanda_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [8]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)

[11, 12, 13, 15, 17, 18, 19, 20, 21, 22, 525, 487, 883, 902, 993, 995, 996, 997, 998, 999, 1000, 1001, 1003, 1610, 1622, 1746]


In [9]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.vivanda.com.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 11, start = 450, end = 499
status_code = 200
cat = 12, start = 400, end = 449
status_code = 200
cat = 13, start = 600, end = 649
status_code = 500
cat = 15, start = 50, end = 99
status_code = 200
cat = 17, start = 1950, end = 1999
status_code = 200
cat = 18, start = 500, end = 549
status_code = 500
cat = 19, start = 350, end = 399
status_code = 200
cat = 20, start = 1100, end = 1149
status_code = 200
cat = 21, start = 800, end = 849
status_code = 200
cat = 22, start = 900, end = 949
status_code = 200
cat = 525, start = 750, end = 799
status_code = 200
cat = 487, start = 400, end = 449
status_code = 500
cat = 883, start = 150, end = 199
status_code = 200
cat = 902, start = 50, end = 99
status_code = 200
cat = 993, start = 950, end = 999
status_code = 200
cat = 995, start = 150, end = 199
status_code = 200
cat = 996, start = 650, end = 699
status_code = 200
cat = 997, start = 550, end = 599
status_code = 200
cat = 998, start = 550, end = 599
status_code = 200
cat 

In [10]:
print(len(products))

13829


In [11]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [12]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

35.39
3000-01-02T05:00:00Z
35.39
35.39


In [13]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "101124703",
    "productName": "Aceite de Sacha Inchi GATTI Extra Virgen Botella 250ml",
    "brand": "GATTI",
    "brandId": 66821,
    "brandImageUrl": null,
    "linkText": "aceite-de-sacha-inchi-gatti-extra-virgen-botella-250ml",
    "productReference": "20462996",
    "productReferenceCode": "20462996",
    "categoryId": "247",
    "productTitle": "Aceite de Sacha Inchi GATTI Extra Virgen Botella 250ml | Vivanda",
    "metaTagDescription": "\u00a1Compra online Aceite de Sacha Inchi GATTI Extra Virgen Botella 250ml al mejor precio en vivanda.com.pe desde donde est\u00e9s!",
    "releaseDate": "2024-10-24T00:00:00Z",
    "clusterHighlights": {},
    "productClusters": {},
    "searchableClusters": {},
    "categories": [
        "/Abarrotes/Aceite/Aceites Especiales/",
        "/Abarrotes/Aceite/",
        "/Abarrotes/"
    ],
    "categoriesIds": [
        "/17/57/247/",
        "/17/57/",
        "/17/"
    ],
    "link": "https://www.vivanda.com.pe/aceite-de-s

In [14]:
# print(json.dumps(products, indent=4))

In [15]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [16]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [17]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [18]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

productId = 100215830
productName = Nectar Naranja ECOFRESH Botella 3.8L
brand = ECOFRESH
brandId = 72216
brandImageUrl = None
categoryId = 1672
productTitle = Nectar Naranja ECOFRESH Botella 3.8L | Vivanda
metaTagDescription = ¡Compra online Nectar Naranja ECOFRESH Botella 3.8L al mejor precio en vivanda.com.pe desde donde estés!
categories = ['/Frutas y Verduras/Frutas/Jugos Naturales/', '/Frutas y Verduras/Frutas/', '/Frutas y Verduras/']
categoriesIds = ['/11/24/1672/', '/11/24/', '/11/']
link = https://www.vivanda.com.pe/jugo-nectar-de-naranja-ecofresh-botella-3-8l/p
Vendido por = None
Descripción del producto = None
Presentación = None
Presentación/Empaque = ['Botella']
Pack = None
Porción Sugerida = None
Azúcares Por Porción = None
Carbohidratos Por Porción = None
Descripción Nutricional = None
Grasas Por Porción = None
Advertencias de consumo = None
itemId = 10338138
ean = 7752705002379
imageUrl = https://vivanda.vteximg.com.br/arquivos/ids/410323/20237284.jpg?v=638025870212830

In [19]:
for product in products:
  new_product = get_product_data(product, fecha_hora_para_base_de_datos)
  df.loc[len(df)] = new_product

In [20]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
11208,100216796,Juego de Pets KAILE ZHI K-1011 Surtido,KAILE ZHI,36364417,None,1386,Juego de Pets KAILE ZHI K-1011 Surtido | Vivanda,¡Compra online Juego de Pets KAILE ZHI K-1011 ...,[/Juguetes y Juegos/Juguetería/Otros Juguetes ...,"[/999/1058/1386/, /999/1058/, /999/]",...,2200202623085,https://vivanda.vteximg.com.br/arquivos/ids/41...,NaN,False,0,0.0,0.0,0.0,None,2024-11-13 14:40:12
6285,47642,Set de Tápers VIVA HOME Plastico 7un,VIVA HOME,72384,None,1225,Set de Tápers VIVA HOME Plastico 7un | Vivanda,¡Compra online Set de Tápers VIVA HOME Plastic...,[/Decohogar/Menaje de Cocina/Tapers y Contened...,"[/22/1029/1225/, /22/1029/, /22/]",...,2200000194503,https://vivanda.vteximg.com.br/arquivos/ids/21...,22.9,True,99999,22.9,22.9,22.9,3000-01-02T00:00:00Z,2024-11-13 14:40:12
8593,100216488,Biberón Natural PHILIPS AVENT 125ML,PHILIPS AVENT,36364747,None,1108,Biberón Natural PHILIPS AVENT 125ML | Vivanda,¡Compra online Biberón Natural PHILIPS AVENT 1...,[/Bebé e Infantil/Alimentación del Bebé/Bibero...,"[/993/1005/1108/, /993/1005/, /993/]",...,8710103875666,https://vivanda.vteximg.com.br/arquivos/ids/40...,NaN,False,0,0.0,0.0,0.0,None,2024-11-13 14:40:12
2013,30151,Salsa DELGA-C César Libre de Grasa Botella 443ml,DELGA-C,28260806,None,280,Salsa DELGA-C César Libre de Grasa Botella 443...,¡Compra online Salsa DELGA-C César Libre de Gr...,"[/Abarrotes/Salsas, Cremas y Condimentos/Aliño...","[/17/64/280/, /17/64/, /17/]",...,746242350441,https://vivanda.vteximg.com.br/arquivos/ids/38...,11.9,True,100,11.9,11.9,11.9,3000-01-02T00:00:00Z,2024-11-13 14:40:12
5720,32830,Tenedores Descartables DARNEL Blanco Paquete 20un,DARNELL PERU,72027,None,410,Tenedores Descartables DARNEL Blanco Paquete 2...,¡Compra Online Tenedores Descartables DARNEL B...,"[/Limpieza/Limpieza de Cocina/Menaje, Utensili...","[/21/97/410/, /21/97/, /21/]",...,645667176585,https://vivanda.vteximg.com.br/arquivos/ids/42...,3.9,True,100,3.9,3.9,3.9,3000-01-02T05:00:00Z,2024-11-13 14:40:12


In [21]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
3289,37323,Ketchup LIBBY'S Doypack 380g,LIBBY S,283,https://www.vivanda.com.pe/ketchup-libbys-doyp...,9.6,0
12769,100540526,Gin HENDRICK'S Flora Adora Botella 700ml,HENDRICK´S,1763,https://www.vivanda.com.pe/gin-hendricks-flora...,134.9,100
7738,46003,Comida para Perros DOG CHOW Adultos Cena de Ca...,DOG CHOW,1694,https://www.vivanda.com.pe/comida-para-perros-...,4.9,100
4191,100087044,Serum GARNIER Express Aclara Frasco 30ml,GARNIER,362,https://www.vivanda.com.pe/serum-garnier-expre...,65.9,100
8209,100984129,Pañal Super Premium BABYSEC Talla XG Paquete 60un,BABYSEC,1138,https://www.vivanda.com.pe/panal-super-premium...,58.9,10
2602,100086584,Lentejas BELL'S Bolsa 500g,BELL'S,310,https://www.vivanda.com.pe/lentejas-bells-bols...,5.5,100
9018,37367,Vaso de Transición sin Boquilla DR. BROWN´S Ch...,DR. BROWN´S,1108,https://www.vivanda.com.pe/vaso-de-transicion-...,0.0,0
9548,4781,Infusiones WAWASANA Té verde limón Caja 20Un,WAWASANA,1294,https://www.vivanda.com.pe/infusiones-wawasana...,9.6,10


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13829 entries, 0 to 13828
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  13829 non-null  object 
 1   productName                13829 non-null  object 
 2   brand                      13829 non-null  object 
 3   brandId                    13829 non-null  int64  
 4   brandImageUrl              0 non-null      object 
 5   categoryId                 13829 non-null  object 
 6   productTitle               13829 non-null  object 
 7   metaTagDescription         13829 non-null  object 
 8   categories                 13829 non-null  object 
 9   categoriesIds              13829 non-null  object 
 10  link                       13829 non-null  object 
 11  Vendido por                0 non-null      object 
 12  Descripción del producto   13829 non-null  object 
 13  Presentación               87 non-null     object 


In [23]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.vivanda.com.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [24]:
df['product_uri_json'].sample(n=5)

11508    https://www.vivanda.com.pe/api/catalog_system/...
7997     https://www.vivanda.com.pe/api/catalog_system/...
13443    https://www.vivanda.com.pe/api/catalog_system/...
234      https://www.vivanda.com.pe/api/catalog_system/...
11929    https://www.vivanda.com.pe/api/catalog_system/...
Name: product_uri_json, dtype: object

In [25]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [26]:
df.iloc[12000]

productId                                                                43539
productName                  Lápices de Grafito ARTESCO FantaYESa 2B Paquet...
brand                                                                  ARTESCO
brandId                                                               10000001
brandImageUrl                                                             None
categoryId                                                                1423
productTitle                 Lápices de Grafito ARTESCO FantaYESa 2B Paquet...
metaTagDescription           ¡Compra online Lápices de Grafito ARTESCO Fant...
categories                   [/Librería y Oficina/Utiles Escolares y de Ofi...
categoriesIds                          [/1000/1063/1423/, /1000/1063/, /1000/]
link                         https://www.vivanda.com.pe/lapices-de-grafito-...
Vendido por                                                               None
Descripción del producto                            